川村さん作成

# 3.3 動的閾値の計算

**スニペット3.1 日次ボラティリティ推定**

日中の推定ポイントで日次ボラティリティを計算

ローリング指数加重標準偏差（span0日の期間を適用）

In [ ]:
def getDailyVol(close, span0=100):
  # 日次ボラティリティ、close(株価系列)に従いインデックス再作成
  df0 = close.index.searchsorted(close.index - pd.Timedelta(days=1))
      # ソートされた配列の中の特定の項目のインデックスを探す
  df0 = df0[df0>0]
  df0 = pd.Series(close.index[df0-1], index=close.index[close.shape[0]-df0.shape[0]:])
      # インデックス付きの1次元配列を作成

  # 日次リターン
  df0 = close.loc[df0.index] / close.loc[df0.values].values -1
  df0 = df0.ewm(span=span0).std()
      # 平均・標準偏差・分散などの統計量を計算するための指数関数(EW)を返す
  return df0

# 3.4 トリプルバリア法

**スニペット3.2 トリプルバリア法**

受け取る引数↓
*   **close** 価格のpandas Series
*   **events** 次の列を含むpandas DataFrame
 *   **t1** 垂直バリアのタイムスタンプ
        np.nanの場合、垂直バリアはない
 *   **trgt** 水平バリアの幅の単位
*   **ptSl** 2つの非負float値のリスト
 *   **ptSl[0]** 上部バリアの幅を設定するためにtrgtに乗算する係数
 0の場合、上部バリアはない
 *   **ptSl[1]** 下部バリアの幅を設定するためにtrgtに乗算する係数
 0の場合、下部バリアはない
*   **molecule** シングルスレッドによって処理されるイベントインデックスのサブセットを含むリスト


In [ ]:
def applyPtSlOnT1(close, events, ptSl, molecule):
  # t1(イベント終了)前に行われた場合は、ストップロス／利食いを実施
  events_ = events.loc[molecule]
  out = events_[['t1']].copy(deep=True)
  if ptSl[0]>0 : pt=ptSl[0]*events_['trgt']
  else : pt=pd.Series(index=events.index) # NaNs
  if ptSl[1]>0 : pt=ptSl[1]*events_['trgt']
  else : sl=pd.Series(index=events.index) # NaNs
  for loc,t1 in events_['t1'].fillna(close.index[-1]).iteritems():
    df0 = close[loc:t1] # 価格経路
    df0 = (df0/close[loc]-1)*events_.at[loc,'side'] # リターン
    # ストップロスの最短タイミング
    out.loc[loc,'sl'] = df0[df0<sl[loc]].index.min()
    # 利食いの最短タイミング
    out.loc[loc,'pt'] = df0[df0>pt[loc]].index.min()
  return out

# 3.5 サイド（side、買いか売りか）とサイズ（size）の学習

**スニペット3.3 最初にバリアに接触する時間の取得**

**getEvents** 最初のバリアに触れる時間を見つける関数

受け取る因数↓

*   **close** 価格のpandas Series
*   **tEvents** すべてのトリプルバリアを生成するタイムスタンプを含むpandasのタイムインデックス（第2章2.5節で説明）
*   **ptSl** 2つのバリアの幅を設定する非負float値
 0の場合、それぞれの水平バリアを無効にする
*   **t1** 垂直バリアのタイムスタンプのpandas Series
 垂直バリアを無効にする場合、Falseを渡す
*   **trgt** 絶対リターンで表現されたターゲットのpandas Series
*   **minRet** トリプルバリアの検索を実行するために必要な最小目標リターン
*   **numThreads** 関数によって同時に使用されているスレッドの数

**mpPandasObj** マルチプロセッシングエンジンを呼び出す（第20章で詳しく説明）

**applyPtSlOnT1** 各バリアが触れられたときのタイムスタンプがあれば返す

出力されるDataFrameの列↓
*   **t1** 最初のバリアに触れたときのタイムスタンプ
*   **trgt** 水平バリアを生成するために使用されたターゲット

In [ ]:
def getEvents(close, tEvents, ptSl, trgt, minRet, numThreads, t1=False):
  #1) ターゲットの定義
  trgt = trgt.loc[tEvents]
  trgt = trgt[trgt>minRet] # minRet
  #2) t1(最大保有期間)の定義
  if t1 is False: t1 = pd.Series(pd.NaT, index=tEvents)
  #3) イベントオブジェクトを作成し、t1にストップロスを適用
  side_ = pd.Series(1., index=trgt.index)
  events = pd.concat({'t1':t1, 'trgt':trgt, 'side':side_}, axis=1).dropna(subset=['trgt'])
  df0 = mpPandasObj(func=applyPtSlOnT1, pdObj=('molecule', events.index), numThreads=numThreads, close=close, events=events, ptSl=[ptSl,ptSl])
  # pd.min は nan を無視する
  events['t1'] = df0.dropna(how='all').min(axis=1)
  events = events.drop('side', axis=1)
  return events

**スニペット3.4 垂直バリアの追加** ※エラー

tEventsの各インデックスに対し、numDaysの日数後または直後の、次のプライスバーのタイムスタンプを求める

求めた垂直バリアは、getEventsのオプション引数t1として渡すことができる

In [ ]:
t1 = close.index.searchsorted(tEvents+pd.Timedelta(days=numDays))
t1 = t1[t1<close.shape[0]]
t1 = pd.Series(close.index[t1], index=tEvents[:t1.shape[0]]) # 終了時に NaNs

**スニペット3.5 サイドとサイズのラベルづけ**

getBins関数で観測値にラベルづけ

出力されるDataFrameの列↓
*   **ret** 最初にバリアに接触した時点で実現したリターン
*   **bin** ラベル｛-1, 0, 1｝

In [ ]:
def getBins(events,close):
  #1) イベント発生時の価格
  events_ = events.dropna(subset=['t1'])
  px = events_.index.union(events_['t1'].values).drop_duplicates()
  px = close.reindex(px,method='bfill')
  #2) out オブジェクトを生成
  out = pd.DataFrame(index=events_.index)
  out['ret'] = px.loc[events_['t1'].values].values/px.loc[events_.index]-1
  out['bin'] = np.sign(out['ret'])
  return out

# 3.6 メタラベリング

**スニペット3.6 メタラベリングを組み込むためのgetEventsの拡張**

**getEvents** 最初のバリアに触れる時間を見つける関数

受け取る因数↓

*   **close** 価格のpandas Series
*   **tEvents** すべてのトリプルバリアを生成するタイムスタンプを含むpandasのタイムインデックス（第2章2.5節で説明）
*   **ptSl** 2つのバリアの幅を設定する非負float値
 0の場合、それぞれの水平バリアを無効にする
*   **t1** 垂直バリアのタイムスタンプのpandas Series
 垂直バリアを無効にする場合、Falseを渡す
*   **trgt** 絶対リターンで表現されたターゲットのpandas Series
*   **minRet** トリプルバリアの検索を実行するために必要な最小目標リターン
*   **numThreads** 関数によって同時に使用されているスレッドの数

新しく追加するオプション引数↓
*   **side** 1次モデルによって決定されたベットのサイド

In [ ]:
def getEvents(close, tEvents, ptSl,trgt, minRet, numThreads, t1=False, side=None):
  #1) ターゲットの定義
  trgt = trgt.loc[tEvents]
  trgt = trgt[trgt>minRet] # minRet
  #2) t1(最大保有期間)の定義
  if t1 is False: t1 = pd.Series(pd.NaT, index=tEvents)
  #3) イベントオブジェクトを作成し、t1にストップロスを適用
  if side is None: side_, ptSl_ = pd.Series(1., index=trgt.index), [ptSl[0], ptSl[0]]
  else: side_, ptSl_ = side.loc[trgt.index], ptSl[:2]
  events = pd.concat({'t1':t1, 'trgt':trgt, 'side':side_}, axis=1).dropna(subset=['trgt'])
  df0 = mpPandasObj(func=applyPtSlOnT1,pdObj=('molecule', events.index), numThreads=numThreads, close=inst['Close'], events=events, ptSl=ptSl_)
  # pd.min は nan を無視する
  events['t1'] = df0.dropna(how='all').min(axis=1)
  if side is None: events=events.drop('side', axis=1)
  return events

**スニペット3.7 メタラベリングを組み込むためのgetBinsの拡張**

In [ ]:
def getBins(events,close):
  '''
  イベント結果を計算する(もしあれば、ロング化ショートかの情報を含む)。
  eventsは次の情報を保持する DataFrame:
  -events.indexはイベント開始時間
  -events['t1']はイベント終了時間
  -events['trgt']はイベントのターゲット
  -events['side'](オプション)はアルゴのポジションサイドを意味する
  ケース1: ('side'がeventsに含まれていない場合): binは(-1,1)のいずれか <-価格変動によるラベル
  ケース2: ('side'がeventsに含まれている場合): binは(0,1)のいずれか <-pn1(メタラベリング)によるラベル
  '''
  #1) events発生時の価格
  events_ = events.dropna(subset=['t1'])
  px = events_.index.union(events_['t1'].values).drop_suplicates()
  px = close.reindex(px, method='bfill')
  #2) outオブジェクトを生成
  out = pd.DataFrame(index=events_.index)
  out['ret'] = px.loc[events_['t1'].values].values/px.loc[events_.index]-1
  # メタラベリング
  if 'side' in events_: out['ret']*=events_['side']
  out['bin'] = np.sign(out['ret'])
  # メタラベリング
  if 'side' in events_: out.loc[out['ret']<=0, 'bin'] = 0
  return out

# 3.9 不要なラベルの削除

**スニペット3.8 出現回数が少ないラベルの削除**

*   **dropLabels** 残りのクラスが2つしかない場合を除き、比率minPct以下の出現頻度のクラスに関連する観測値を削除する

In [ ]:
def dropLabels(events, minPct=.05):
  # ウェイトを適用し、不十分なラベルを削除する例
  while True:
    df0 = events['bin'].value_counts(normalize=True)
    if df0.min()>minPct or df0.shape[0]<3: break
    print('dropped label'), df0.argmin(), df0.min()
    events = events[events['bin']!=df0.argmin()]
  return events